<a href="https://colab.research.google.com/github/shivkumarganesh/Advance-Deep-Learning/blob/main/Assignment%203/NER_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip3 install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-dqs4lta9
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-dqs4lta9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [34]:
from datasets import load_dataset

dataset = load_dataset("afrikaans_ner_corpus")

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/954 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.7M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset afrikaans_ner_corpus downloaded and prepared to /root/.cache/huggingface/datasets/afrikaans_ner_corpus/afrikaans_ner_corpus/1.0.0/f26d2b25bd3f8cfc4421e00d43d47f23738c1c6717992d4f9913f213146fd7c4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [35]:
from transformers import AutoTokenizer,AutoConfig, AutoModelForTokenClassification
MODEL_NAME = 'bert-base-german-cased' 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

loading configuration file https://huggingface.co/bert-base-german-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/98877e98ee76b3977d326fe4f54bc29f10b486c317a70b6445ac19a0603b00f0.1f2afedb22f9784795ae3a26fe20713637c93f50e2c99101d952ea6476087e5e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

loading file https://huggingface.co/bert-base-german-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transfor

In [40]:
def tokenize_function_doc(example):
  return tokenizer(example["document"], padding="max_length", truncation=True)

def tokenize_function_summary(example):
  return tokenizer(example["summary"], padding="max_length", truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function_doc, batched=True)
tokenized_datasets = raw_datasets.map(tokenize_function_summary, batched=True)

  0%|          | 0/205 [00:00<?, ?ba/s]

In [42]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [43]:
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=2)

loading configuration file https://huggingface.co/bert-base-german-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/98877e98ee76b3977d326fe4f54bc29f10b486c317a70b6445ac19a0603b00f0.1f2afedb22f9784795ae3a26fe20713637c93f50e2c99101d952ea6476087e5e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

loading weights file https://huggingface.co/bert-base-german-cased/resolve/main/pytorch_model.bin from cache at /root/.cache/hugg

In [44]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [50]:
from transformers import Trainer
import numpy as np
from datasets import load_metric

mertic = load_metric("accuracy")

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [51]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return mertic.compute(predictions=predictions, references=labels)

In [52]:
trainer = Trainer(
    model= model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: document, summary, id.
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375


In [1]:
trainer.evaluate()

NameError: ignored